In [2]:
import cv2  #pour le traitement d'images et la vision par ordinateur
import face_recognition   #une bibliothèque spécialisée dans la reconnaissance faciale.
import os   # manipuler les fichiers et les répertoires
import glob  # rechercher des fichiers et des répertoires en utilisant des motifs de recherche
import numpy as np   # pour le calcul numérique en Python
import matplotlib.pyplot as plt  #bibliothèque de visualisation en 2D

ModuleNotFoundError: No module named 'face_recognition'

In [3]:

class Facerec: # créée une classe pour encapsuler les fonctionnalités liées à la reconnaissance faciale.
    def __init__(self):
        self.known_faces_encodings = []   #pour stocker les encodages des visages connus.
        self.known_faces_names = []   #pour stocker les noms des visages connus.
        self.frame_resizing = 0.25   # coefficient de redimensionnement des images de trame

    def apply_filters(self, img):
        # Convert to grayscale for Laplacian filter
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray_img], [0], None, [256], [0,256])
        #Normalisation de l'image en niveaux de gris
        Imin, Imax = np.min(gray_img), np.max(gray_img)
        img_normalized = (255 / (Imax - Imin)) * (gray_img - Imin)
        #Sharpening using the provided kernel
        kernel = np.array(([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]), dtype=np.float32)
        img_sharpened = cv2.filter2D(img_normalized, -1, kernel) #filtre de renforcement des contours en utilisant une convolution avec un noyau prédéfini
        img_sharpened = np.clip(img_sharpened, 0, 255) #np.clip est utilisée pour s'assurer que les valeurs restent dans la plage 0-255.
        gauss_blur = cv2.GaussianBlur(img_sharpened,(3,3),1) #Un filtre de flou gaussien est appliqué à l'image (reduire le bruit)
        #Filtres de Sobel (détection des contours)
        noy_sbx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
        noy_sby = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
        horizs = cv2.filter2D(gauss_blur,-1,noy_sby)
        vertics = cv2.filter2D(gauss_blur,-1,noy_sbx)
        #Calcul du module du gradient Sobel
        module_sobel = np.sqrt(np.square(vertics.astype(np.float32)) + np.square(horizs.astype(np.float32)))
        mod_sb = np.uint8(module_sobel)
        #Amélioration de l'image d'origine avec le gradient( en ajoutant 50% du module du gradient Sobel.)
        enhanced_image = cv2.addWeighted(gray_img, 1, mod_sb, 0.5, 0) # Combinaison

        return img_normalized,img_sharpened, mod_sb ,enhanced_image

    def display_images(self, normalized, sharpened, edge_detection):
      # Display original and preprocessed images
        plt.figure(figsize=(10, 8))


        plt.subplot(2, 4, 1), plt.imshow(normalized, cmap='gray')
        plt.title('Normalized'), plt.axis('off')

        plt.subplot(2, 4, 2), plt.imshow(sharpened, cmap='gray')
        plt.title('Sharpened'), plt.axis('off')

        plt.subplot(2, 4, 3), plt.imshow(edge_detection, cmap='gray')
        plt.title('edge detection'), plt.axis('off')

        plt.show()

    def process_images_in_folder(self, folder_path):
        # Iterate through all images in the folder
        for filename in os.listdir(folder_path):  #os.listdir(folder_path) pour obtenir la liste des fichiers dans le dossier.
            if filename.endswith(('.png', '.jpg', '.jpeg','.JPG')): #vérification s'il a une extension
                # Read the image
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)

                # Apply filters
                img_normalized, img_sharpened,mod_sb ,enhanced_image  = self.apply_filters(img)

                # Display images
                self.display_images(img_normalized, img_sharpened, mod_sb)

    def load_encoding_images(self, images_path):  # Méthode pour charger les images et leurs encodages.
        images_path = glob.glob(os.path.join(images_path, "*.*")) #Utilise glob.glob pour obtenir une liste de chemins d'images dans le répertoire spécifié (images_path).
        print("{} encoding images found.".format(len(images_path))) #Affiche le nombre d'images trouvées dans le répertoire

        for img_path in images_path:
            img = cv2.imread(img_path)

            # Appliquer les filtres à chaque image.
            img_normalized, img_sharpened,mod_sb ,enhanced_image  = self.apply_filters(img)
            rgb_img = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB)#Convertit l'image en BGR

            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)#pour obtenir le nom du fichier et son extension à partir du chemin d'accès de l'image.

            # Get encoding
            face_encodings = face_recognition.face_encodings(rgb_img)#pour obtenir les encodages des visages dans l'image.

            if face_encodings:
                #Stocker le nom du fichier et les encodages de visage dans les listes
                self.known_faces_encodings.extend(face_encodings)
                self.known_faces_names.extend([filename] * len(face_encodings))
                #Duplique le nom du fichier pour chaque visage détecté.
            else:
                # Handle the case when no face encodings are found
                print("No face encodings found for image:", img_path)

        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)   #Redimensionne la trame d'entrée (frame) en utilisant la fonction cv2.resize
                                                                                                  # fx et fy déterminent le facteur de redimensionnement en largeur et en hauteur respectivement.

        # Convert the image from BGR color to RGB color
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)  #détecter les emplacements des visages dans la trame
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)   #extraire les encodages faciaux des visages détectés
                                                                                             #Ces encodages sont utilisés pour comparer les visages détectés avec les visages connus stockés précédemment

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face
            matches = face_recognition.compare_faces(self.known_faces_encodings, face_encoding)
            name = "Unknown"

            #Calcule les distances entre l'encodage facial actuel et les encodages faciaux des visages connus.
            face_distances = face_recognition.face_distance(self.known_faces_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)  # Trouve l'index du visage connu avec la plus petite distance par rapport au visage actuellement détecté.
            if matches[best_match_index]:
                name = self.known_faces_names[best_match_index]  #met à jour le nom du visage détecté avec le nom du visage connu correspondant.
            face_names.append(name)   #Ajoute le nom du visage à la liste face_names

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing  #Ajuste les coordonnées des emplacements des visages en fonction du facteur de redimensionnement
        return face_locations.astype(int), face_names  # Retourne un tuple contenant les emplacements des visages en tant que tableau d'entiers et les noms des visages détectés.




In [4]:
#Display result images
facerec_instance = Facerec()
folder_path = "images/"
facerec_instance.process_images_in_folder(folder_path)

NameError: name 'os' is not defined

In [ ]:

sfr = Facerec()
sfr.load_encoding_images("images/")


In [ ]:
cap = cv2.VideoCapture(0) #Initialise la capture vidéo en utilisant la webcam
while True: #boucle pour capturer des images en continu.
    #Lit une image à partir de la webcam
    ret, frame = cap.read()  #La variable frame contient l'image capturée.
    #Detection des visages
    face_locations,face_names =sfr.detect_known_faces(frame) #pour détecter les visages connus dans l'image
    for face_loc,name in zip(face_locations,face_names):
      #Boucle à travers chaque paire de coordonnées de visage et nom associé.
        y1, x2, y2, x1 = face_loc[0] , face_loc[1], face_loc[2], face_loc[3] # pour obtenir les positions des coins du rectangle délimitant le visage.
        cv2.putText(frame,name,(x1,y1-10),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,200),2) #Ajoute le nom du visage au cadre à une position spécifique au-dessus du visage.
        cv2.rectangle(frame,(x1,y1),(x2, y2),(0,0,200),4) #pour Dessiner un rectangle autour du visage sur l'image capturée.
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1)
    if key == 27:  #Si la touche pressée est la touche Échap (code 27), cela casse la boucle (break), mettant fin à la capture vidéo.
        break

cap.release()  #Libère la ressource de capture vidéo.
cv2.destroyAllWindows() #Ferme toutes les fenêtres d'affichage OpenCV.